In [1]:
import pandas as pd
import requests
from sqlalchemy import create_engine
import zipfile
from io import BytesIO

In [2]:
api = "http://api.16g5.com"
json = requests.get(api).json()

file_list = []

for url in json["vod"]:
    resp = requests.get(url)

    filename = url[url.rindex('/') + 1:]
    with open(filename, 'wb') as file:
        file.write(resp.content)

    zf = zipfile.ZipFile(filename)
    zf.extractall()
    file_list.extend(zf.namelist())
    zf.close()

file_list

['vod1.txt', 'vod2.txt', 'vod3.txt', 'vod4.txt', 'vod5.txt']

In [3]:
engine = create_engine('sqlite:///video.db')

for i, file in enumerate(file_list):
    df = pd.read_csv(file, sep="\|-\|", engine='python', names=['ID', 'CLASS', 'TITLE', 'PIC', 'URL', 'DATE'])
    df.head()

    if_exists = 'replace' if i == 0 else 'append'
    df.to_sql('vod', engine, if_exists=if_exists)


In [4]:
with engine.connect() as conn, conn.begin():
    data = pd.read_sql_table('vod', conn)

data.head()

ValueError: invalid literal for int() with base 10: 'https://pic.cnljpic.com/upload/vod/20210213-1/258a22003101dbb9915fb54bd7a315ae.jpg'